In [1]:
import torch
from torch import Tensor
import numpy as np

# 1 Multiple views of a storage

In [2]:
t0 = Tensor(13,13).fill_(3.0)

t0.narrow(0,0,3).fill_(1.0)
t0.narrow(0,5,3).fill_(1.0)
t0.narrow(0,10,3).fill_(1.0)

t0.narrow(1,0,3).fill_(1.0)
t0.narrow(1,5,3).fill_(1.0)
t0.narrow(1,10,3).fill_(1.0)

t0.narrow(0,1,1).fill_(2.0)
t0.narrow(0,6,1).fill_(2.0)
t0.narrow(0,12,1).fill_(2.0)

t0.narrow(1,1,1).fill_(2.0)
t0.narrow(1,6,1).fill_(2.0)
t0.narrow(1,12,1).fill_(2.0)

t0


    1     2     1     1     1     1     2     1     1     1     1     1     2
    2     2     2     2     2     2     2     2     2     2     2     2     2
    1     2     1     1     1     1     2     1     1     1     1     1     2
    1     2     1     3     3     1     2     1     3     3     1     1     2
    1     2     1     3     3     1     2     1     3     3     1     1     2
    1     2     1     1     1     1     2     1     1     1     1     1     2
    2     2     2     2     2     2     2     2     2     2     2     2     2
    1     2     1     1     1     1     2     1     1     1     1     1     2
    1     2     1     3     3     1     2     1     3     3     1     1     2
    1     2     1     3     3     1     2     1     3     3     1     1     2
    1     2     1     1     1     1     2     1     1     1     1     1     2
    1     2     1     1     1     1     2     1     1     1     1     1     2
    2     2     2     2     2     2     2     2     2     2    

# 2 Eigendecomposition

In [3]:
m_mat = Tensor(20,20).normal_()
m_dia = torch.diag(torch.arange(1,m_mat.size(0)+1))

prod = m_mat.inverse().mm(m_dia).mm(m_mat)

v, _ = torch.eig(prod)
print(v.narrow(1,0,1).squeeze().sort()[0])


  1.0000
  2.0000
  3.0000
  4.0000
  5.0000
  6.0000
  7.0000
  8.0000
  9.0000
 10.0000
 11.0000
 12.0000
 13.0000
 14.0000
 15.0000
 16.0000
 17.0000
 18.0000
 19.0000
 20.0000
[torch.FloatTensor of size 20]




# 3 Flops per second

In [4]:
import time

In [9]:
d = 5000
m_1 = Tensor(d, d).normal_()
m_2 = Tensor(d, d).normal_()

time_t0 = time.perf_counter()
m_prod = m_1.mm(m_2)
time_t1 = time.perf_counter()

time_diff = time_t1 - time_t0
print(f'time: {time_diff}')

time: 1.3851563069911208


Estimation of floating point products per second (in billions - 1,000,000,000):

In [13]:
d**3/time_diff*1e-9

90.24252307779535

# 4 Playing with strides

Slow function:

In [14]:
def mul_row(m):
    m2 = torch.Tensor(m.size())
    for i in range(0, m.size()[0]):
        for j in range(0, m.size()[1]):
            m2[i,j] = m[i,j]*(i+1)
    return m2

In [15]:
# Verify it's correct
m = torch.Tensor(4,8).fill_(2.0)
m2 = mul_row(m)
print(m2)


    2     2     2     2     2     2     2     2
    4     4     4     4     4     4     4     4
    6     6     6     6     6     6     6     6
    8     8     8     8     8     8     8     8
[torch.FloatTensor of size 4x8]



In [16]:
# Compute time
m = torch.Tensor(10000,400).fill_(2.0)

time_t0 = time.perf_counter()
m2 = mul_row(m)
time_t1 = time.perf_counter()

time_diff = time_t1 - time_t0
print(f'time: {time_diff}')

time: 3.7823165050067473


Fast function:

In [25]:
def mul_row_fast(m):
    #r = torch.arange(1,m.size()[0]+1).view(-1,1)
    d = m.size(0)
    r = torch.arange(1, d + 1).view(-1, 1)
    return m.mul(r)

In [27]:
# Verify it's correct
m = torch.Tensor(4,8).fill_(2.0)
m2 = mul_row_fast(m)
print(m2)


    2     2     2     2     2     2     2     2
    4     4     4     4     4     4     4     4
    6     6     6     6     6     6     6     6
    8     8     8     8     8     8     8     8
[torch.FloatTensor of size 4x8]



In [28]:
# Compute time
m = torch.Tensor(10000,400).fill_(2.0)

time_t0 = time.perf_counter()
m2 = mul_row_fast(m)
time_t1 = time.perf_counter()

time_diff = time_t1 - time_t0
print(f'time: {time_diff}')

time: 0.01671235598041676
